<a href="https://colab.research.google.com/github/Skkuhodomo/private_LLM/blob/main/2_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리 다운로드

In [ ]:
!pip install matplotlib numpy pylzma ipykernel

In [ ]:
!pip install torch torchvision torchaudio

In [7]:
import torch
with open('story.txt', 'r', encoding = 'utf-8') as f:
  text = f.read()
chars = sorted(set(text))

In [8]:
string_to_int =  {ch:i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([16, 24, 21,  1, 29, 17, 28, 21,  1, 30, 22,  1, 35, 24, 21,  1, 19, 24,
        25, 27, 20,  1, 38, 17, 34,  1, 12, 36, 29, 24, 30,  5,  1, 10, 21,  1,
        38, 17, 34,  1, 22, 17, 28, 30, 36, 34,  1, 22, 30, 33,  1, 24, 25, 34,
         1, 32, 36, 25, 21, 35,  1, 17, 29, 20,  1, 19, 17, 27, 28,  1, 20, 21,
        28, 21, 17, 29, 30, 33,  5,  1, 10, 30, 38, 21, 37, 21, 33,  4,  1, 35,
        24, 21, 33, 21,  1, 38, 17, 34,  1, 17])


## Train and validation splits

In [11]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [12]:
block_size = 8

x= train_data[:block_size]
y= train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print('when input is', context, 'target is', target)

when input is tensor([16]) target is tensor(24)
when input is tensor([16, 24]) target is tensor(21)
when input is tensor([16, 24, 21]) target is tensor(1)
when input is tensor([16, 24, 21,  1]) target is tensor(29)
when input is tensor([16, 24, 21,  1, 29]) target is tensor(17)
when input is tensor([16, 24, 21,  1, 29, 17]) target is tensor(28)
when input is tensor([16, 24, 21,  1, 29, 17, 28]) target is tensor(21)
when input is tensor([16, 24, 21,  1, 29, 17, 28, 21]) target is tensor(1)


CPU와 다르게 GPU는 더 효율적으로 해당 작업을 수행할 수 있다. prvateLLM 프로젝트에서는 colab을 이용하므로 별도의 다운로드 없이 쉽게 변경이 가능하다.

In [13]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4

cuda


In [16]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y
x, y = get_batch( 'train')
print('inputs:')
# print(x. shape)
print(x)
print('targets:')
print (y)

inputs: 
tensor([[30,  1, 18, 21, 19, 30, 28, 21],
        [ 1, 24, 17, 33, 20, 38, 17, 33],
        [22, 33, 25, 21, 29, 20, 34,  4],
        [21, 23, 17, 29,  1, 35, 30,  1]], device='cuda:0')
targets:
tensor([[ 1, 18, 21, 19, 30, 28, 21,  1],
        [24, 17, 33, 20, 38, 17, 33, 21],
        [33, 25, 21, 29, 20, 34,  4,  1],
        [23, 17, 29,  1, 35, 30,  1, 34]], device='cuda:0')
